In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20960220
paper_name = 'jayakody_kitagaki_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/10529_2010_437_MOESM1_ESM.xlsx', sheet_name='Table 1', skiprows=7)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 169 x 4


In [7]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [9]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene name, Ratio, Function of the gene products]
Index: []


In [11]:
# Convert data to follow convention (centered on 0, negative = defect)
original_data['data'] = original_data['Ratio']-1

In [12]:
original_data.set_index('ORF', inplace=True)
original_data.index.name = 'orf'

In [13]:
original_data = original_data[['data']].copy()

In [14]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(167, 1)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [16473]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,16473
data_type,value
orf,
YAR037W,-0.134
YBL031W,-0.129
YBR059C,-0.187
YBR092C,-0.168
YBR169C,-0.155


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16473
,data_type,value
gene_id,orf,
119,YBL031W,-0.129
255,YBR059C,-0.187
287,YBR092C,-0.168
363,YBR169C,-0.155
493,YBR299W,-0.145


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id       16473          
data_type        value    valuez
gene_id orf                     
119     YBL031W -0.129 -2.741774
255     YBR059C -0.187 -3.974510
287     YBR092C -0.168 -3.570683
363     YBR169C -0.155 -3.294380
493     YBR299W -0.145 -3.081840

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 20960220...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  2.41it/s]

Updating the data_modified_on field...
